In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [5]:
root_folder = "./tactile_dataset/"
file_name = "final_merged_df_sw500.csv"

df = pd.read_csv(root_folder + file_name)
print(df.head())

y = df['Texture']
X = df.drop(['Texture'], axis=1)

     imu_ax    imu_ay    imu_az    imu_gx    imu_gy    imu_gz    imu_mx  \
0 -0.714646 -3.751385  8.952035  0.271843  0.887275  0.235063 -0.142849   
1  1.325862 -3.764498  8.800300 -0.303642  0.904468  0.343564  0.270401   
2  2.542223 -3.111482  8.754128 -0.245584  0.441520  0.233173  0.519580   
3  2.803708 -2.582370  8.868835 -0.337755  0.003374  0.098259  0.508302   
4  3.854054 -2.298121  8.681546 -0.108013  0.197704  0.046265  0.777799   

     imu_my    imu_mz  Texture     baro  
0 -0.747876  1.757683      1.0  173.848  
1 -0.767744  1.794762      1.0  194.202  
2 -0.632350  1.753249      1.0  196.912  
3 -0.499855  1.791200      1.0  196.594  
4 -0.461740  1.769799      1.0  200.554  


In [6]:
# pre-processing data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder

def scale_data(X_train, X_test):
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test

def encode_target(y_train, y_test):
    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.transform(y_test)
    return y_train, y_test

def encode_target_onehot(y_train, y_test):
    onehot_encoder = OneHotEncoder(sparse_output=False)
    y_train = y_train.values.reshape(-1, 1)
    y_test = y_test.values.reshape(-1, 1)
    y_train = onehot_encoder.fit_transform(y_train)
    y_test = onehot_encoder.transform(y_test)
    return y_train, y_test


In [59]:
delta_thresholds = None
def poisson_encoding(features, num_steps):
    features_repeated = features.unsqueeze(1).repeat(1, num_steps)
    rand_vals = torch.rand_like(features_repeated)
    spikes = (rand_vals < features_repeated).float()
    return spikes

def temporal_contrast_encode(features, num_steps, delta_thresholds):
    diffs = torch.diff(features, dim=0, prepend=features[:1])
    spikes = (torch.abs(diffs) > delta_thresholds).float()
    spikes = spikes.unsqueeze(1).repeat(1, num_steps, 1)
    return spikes

def get_delta_thresholds(features, k):
    diffs = torch.diff(features, dim=0, prepend=features[:1])
    return (torch.std(diffs, dim=0) * k)

def encode_to_spike_train(features, num_steps=50, k=2, delta_thresholds=None):
    features_for_rate_coding = features[:, 9]
    features_for_temporal_contrast = features[:, :9]
    if delta_thresholds is None:
        delta_thresholds = get_delta_thresholds(features_for_temporal_contrast, k)
    print(delta_thresholds)
    spikes_rate_coding = poisson_encoding(features_for_rate_coding, num_steps)
    spikes_temporal_contrast = temporal_contrast_encode(features_for_temporal_contrast, num_steps, delta_thresholds)
    spikes_rate_coding = spikes_rate_coding.unsqueeze(-1)

    return torch.cat([spikes_rate_coding, spikes_temporal_contrast], dim=-1), delta_thresholds

In [67]:
delta_thresholds = None
num_steps = 100
k = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_test = scale_data(X_train, X_test)
y_train, y_test = encode_target(y_train, y_test)

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
X_train, delta_thresholds = encode_to_spike_train(X_train, num_steps=num_steps, k=k)

X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.long).to(device)
X_test = encode_to_spike_train(X_test, num_steps=num_steps, k=k, delta_thresholds=delta_thresholds)[0]

tensor([0.2160, 0.2852, 0.1739, 0.1218, 0.1318, 0.1326, 0.2130, 0.2845, 0.1549])
tensor([0.2160, 0.2852, 0.1739, 0.1218, 0.1318, 0.1326, 0.2130, 0.2845, 0.1549])


In [68]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 32

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [69]:
print(y_train.shape)
print(y_train[:5])

print(X_train.shape)

torch.Size([72532])
tensor([0, 7, 9, 2, 5])
torch.Size([72532, 100, 10])


In [70]:
import snntorch as snn

num_inputs = X_train.shape[2]
num_outputs = 12
num_hidden = 100
beta = 0.9

class SNNModel(nn.Module):
    def __init__(self):
        super(SNNModel, self).__init__()
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden, num_hidden)
        self.lif2 = snn.Leaky(beta=beta)
        self.fc3 = nn.Linear(num_hidden, num_outputs)
        self.lif3 = snn.Leaky(beta=beta)

    def forward(self, x):
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()

        batch_size, num_steps, _ = x.shape
        output_spikes = torch.zeros(batch_size, num_outputs).to(device)
        for step in range(x.size(1)):
            input_step = x[:, step, :]
            cur1 = self.fc1(input_step)
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            cur3 = self.fc3(spk2)
            spk3, mem3 = self.lif3(cur3, mem3)
            output_spikes += spk3
        return output_spikes / num_steps
    

In [71]:
import time

def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        total_correct = 0.0
        start_time = time.time()
        for _, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            total_correct += (outputs.argmax(dim=-1) == labels.argmax(dim=-1)).float().mean()
        elapsed_time = time.time() - start_time
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader):.4f}, Accuracy: {(total_correct / len(train_loader))*100:.2f}, Time: {elapsed_time:.4f} sec")


In [ ]:
net = SNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=5e-3)

train_model(net, train_loader, criterion, optimizer, num_epochs=10)